### TF-IDF SOU Addresses: we use the classic vector space model from information retrieval to find similar State Of the Union addresses.

In [186]:
from __future__ import division
import string
import math

In [221]:
with open("speeches.pkl", "r") as f:
    w = pickle.load(f)
print "Number of Speeches: " + str(len(w))
no_documents = len(w)

Number of Speeches: 226


In [188]:
def clean_and_split(s):
    s = s.encode('utf-8').lower().replace('-',' ').translate(None, string.punctuation)
    s = re.sub('(\r\n)+',' ',s)
    s = re.sub(' +',' ',s.strip())
    return s.split(' ')

In [189]:
i = 0
speeches = []
while (i < no_documents):
    speeches.append(w[i].text)
    i += 1

In [191]:
def word_frequency(word, wordlist):
    return wordlist.count(word)

In [ ]:
from nltk.corpus import stopwords
def rem_stop_words(wordlists):
    for speech in wordlists:
        

In [196]:
def tfidf(speeches):
    boolean = 0 
    wordlists = [clean_and_split(s) for s in speeches]
    idf = inverse_speech_frequencies(wordlists)
    tfidf_speeches = []
    for speech in wordlists:
        speech_tfidf = []
        for word in idf.keys():
            if (word_frequency(word, speech) > 50): #removes words that appear fewer than 50 times 
                tf = sublinear_word_frequency(word, speech)
                speech_tfidf.append(tf * idf[word])
                boolean = 1
        if (boolean):
            tfidf_speeches.append(speech_tfidf)
    return tfidf_speeches

In [192]:
def inverse_speech_frequencies(wordlist):
    idf_values = {}
    all_words_set = set([item for sublist in wordlist for item in sublist])
    for wrd in all_words_set:
        contains_word = map(lambda speech: wrd in speech, wordlist)
        idf_values[wrd] = 1 + math.log(len(wordlist)/(sum(contains_word)))
    return idf_values

def sublinear_word_frequency(word, wordlist):
    return 1 + math.log(wordlist.count(word))

In [197]:
from sklearn.feature_extraction.text import TfidfVectorizer

sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, analyzer = 'word')
sklearn_representation = sklearn_tfidf.fit_transform(speeches)

In [202]:
tfidf_representation = tfidf(speeches)

In [207]:
print ("TDF-IF Vector for 1st Speech: \n")
print sklearn_representation.toarray()[0].tolist()

TDF-IF Vector for 1st Speech: 

[0.0, 0.010256367663686759, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.045417592716779066, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.027592237097620972, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07028091687221068, 0.03876104469400377, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.026123346762627713, 0.014309667754680452, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [206]:
def cosine_similarity(vector1, vector2):
    dot_product = sum(p*q for p,q in zip(vector1, vector2))
    magnitude = math.sqrt(sum([val**2 for val in vector1])) * math.sqrt(sum([val**2 for val in vector2]))
    if not magnitude:
        return 0
    return dot_product/magnitude

In [214]:
skl_tfidf_comparisons = []
for count_0, speech_0 in enumerate(sklearn_representation.toarray()):
    for count_1, speech_1 in enumerate(sklearn_representation.toarray()):
        if (count_0 != count_1):
            skl_tfidf_comparisons.append((cosine_similarity(speech_0, speech_1), count_0, count_1))

In [288]:
def rem_duplicates(skl_tfidf_comparisons):
    i = 0
    new_tfidf_comparisons = []
    for x in skl_tfidf_comparisons:
        if (i % 2 == 0):
            new_tfidf_comparisons.append(x)
        i += 1
    return new_tfidf_comparisons
        
i = 0
diff_presidents = []
sort_tfidf_comparisons = rem_duplicates(sorted(skl_tfidf_comparisons, reverse = True))
print "50 most similar pairs of SOUs given by different Presidents \n"
for x in sort_tfidf_comparisons:
    if (w[x[1]].president != w[x[2]].president):
        diff_presidents.append(x)
        if (i < 50):
            print (x[0], w[x[1]].president, w[x[2]].president, w[x[1]].year, w[x[2]].year);
            i += 1

50 most similar pairs of SOUs given by different Presidents 

(0.45387379427468044, u'Andrew Jackson', u'Martin Van Buren', u'1836', u'1837')
(0.4424983802576889, u'Andrew Jackson', u'Martin Van Buren', u'1836', u'1839')
(0.4415166065803033, u'Chester A. Arthur', u'Rutherford B. Hayes', u'1881', u'1880')
(0.43700625470887633, u'William McKinley', u'William Howard Taft', u'1899', u'1912')
(0.43500867440804153, u'William Howard Taft', u'Theodore Roosevelt', u'1912', u'1907')
(0.4316153685861886, u'Andrew Jackson', u'Martin Van Buren', u'1830', u'1837')
(0.43155149558414707, u'Andrew Jackson', u'Martin Van Buren', u'1832', u'1837')
(0.42993259928903216, u'James Buchanan', u'James K. Polk', u'1858', u'1847')
(0.4286242909098021, u'Andrew Jackson', u'Martin Van Buren', u'1834', u'1837')
(0.4279670946395277, u'James K. Polk', u'Martin Van Buren', u'1845', u'1837')
(0.42630374851368597, u'Andrew Jackson', u'Martin Van Buren', u'1834', u'1839')
(0.4255155295431593, u'Chester A. Arthur', u'Ruth

In [289]:
i = 0
print "50 most similar pairs of SOUs given by the same President \n"
for x in sort_tfidf_comparisons:
    if (i < 50):
        if (w[x[1]].president == w[x[2]].president):
            print (x[0], w[x[1]].president, w[x[2]].president, w[x[1]].year, w[x[2]].year);
            i += 1

50 most similar pairs of SOUs given by the same President 

(0.833963364442439, u'Barack Obama', u'Barack Obama', u'2012', u'2013')
(0.5128483519571347, u'James K. Polk', u'James K. Polk', u'1847', u'1846')
(0.5050283653656397, u'James Buchanan', u'James Buchanan', u'1858', u'1857')
(0.5049412835405339, u'Rutherford B. Hayes', u'Rutherford B. Hayes', u'1880', u'1879')
(0.49562411900957526, u'William J. Clinton', u'William J. Clinton', u'2000', u'1999')
(0.4951890460926244, u'Theodore Roosevelt', u'Theodore Roosevelt', u'1907', u'1905')
(0.49448516750547405, u'William Howard Taft', u'William Howard Taft', u'1910', u'1912')
(0.4926729866943094, u'Dwight D. Eisenhower', u'Dwight D. Eisenhower', u'1956', u'1955')
(0.49146320328813486, u'Theodore Roosevelt', u'Theodore Roosevelt', u'1904', u'1905')
(0.49065459376288983, u'James K. Polk', u'James K. Polk', u'1846', u'1845')
(0.4894695938502923, u'Andrew Johnson', u'Andrew Johnson', u'1868', u'1867')
(0.48865475193479185, u'William Howard Taf

In [257]:
president_sim_list = [{'President_1': 'Andrew Jackson', 'President_2': 'Martin Van Buren', 
                       'Similarity': 0.45387379427468044, 'Total': 1}]
for x in diff_presidents:
    dict_pres_1 = [(idx, comp) for idx, comp in enumerate(president_sim_list) 
     if comp['President_1'] == w[x[1]].president and comp['President_2'] == w[x[2]].president]
    dict_pres_2 = [(idx, comp) for idx, comp in enumerate(president_sim_list) 
     if comp['President_1'] == w[x[2]].president and comp['President_2'] == w[x[1]].president]
    if (not dict_pres_1) and (not dict_pres_2):
        dict_pres = {'President_1': w[x[1]].president, 'President_2': w[x[2]].president, 
                       'Similarity': x[0], 'Total': 1}
        president_sim_list.append(dict_pres)
    if (dict_pres_1):
        president_sim_list[dict_pres_1[0][0]]['Similarity'] += x[0]
        president_sim_list[dict_pres_1[0][0]]['Total'] += 1
        #print president_sim_list[dict_pres_1[0][0]]
    if (dict_pres_2):
        president_sim_list[dict_pres_2[0][0]]['Similarity'] += x[0]
        president_sim_list[dict_pres_2[0][0]]['Total'] += 1
        #print president_sim_list[dict_pres_2[0][0]]
print president_sim_list    

[{'Total': 33, 'Similarity': 13.472193692217779, 'President_2': 'Martin Van Buren', 'President_1': 'Andrew Jackson'}, {'Total': 16, 'Similarity': 6.112121783077517, 'President_2': u'Rutherford B. Hayes', 'President_1': u'Chester A. Arthur'}, {'Total': 16, 'Similarity': 6.128393582171108, 'President_2': u'William Howard Taft', 'President_1': u'William McKinley'}, {'Total': 32, 'Similarity': 12.008812762691901, 'President_2': u'Theodore Roosevelt', 'President_1': u'William Howard Taft'}, {'Total': 16, 'Similarity': 6.395273542423704, 'President_2': u'James K. Polk', 'President_1': u'James Buchanan'}, {'Total': 16, 'Similarity': 6.3425828116798835, 'President_2': u'Martin Van Buren', 'President_1': u'James K. Polk'}, {'Total': 32, 'Similarity': 11.541338142863996, 'President_2': u'Grover Cleveland', 'President_1': u'William McKinley'}, {'Total': 32, 'Similarity': 12.198043736367758, 'President_2': u'James K. Polk', 'President_1': u'Andrew Jackson'}, {'Total': 16, 'Similarity': 5.876481470

In [290]:
president_pairs = []
for x in president_sim_list:
    s_dict = {'President_1': x['President_1'], 'President_2': x['President_2'], 
              'Average': (x['Similarity']/x['Total'])};
    president_pairs.append(s_dict);
i = 0
print "25 most similar pairs of Presidents,  averaging the cosine similarity over all pairs of their SOUs.\n"
for x in sorted(president_pairs, key = lambda pres: pres['Average'], reverse = True):
    if (i < 25):
        print x
    else:
        break
    i += 1

25 most similar pairs of Presidents,  averaging the cosine similarity over all pairs of their SOUs.

{'Average': 0.4082482937035691, 'President_2': 'Martin Van Buren', 'President_1': 'Andrew Jackson'}
{'Average': 0.3997045964014815, 'President_2': u'James K. Polk', 'President_1': u'James Buchanan'}
{'Average': 0.39924197157181024, 'President_2': u'Millard Fillmore', 'President_1': u'Zachary Taylor'}
{'Average': 0.3964114257299927, 'President_2': u'Martin Van Buren', 'President_1': u'James K. Polk'}
{'Average': 0.38905659588179836, 'President_2': u'James K. Polk', 'President_1': u'Millard Fillmore'}
{'Average': 0.3837938108998642, 'President_2': u'Martin Van Buren', 'President_1': u'John Tyler'}
{'Average': 0.38302459888569423, 'President_2': u'William Howard Taft', 'President_1': u'William McKinley'}
{'Average': 0.38264033774228834, 'President_2': u'Zachary Taylor', 'President_1': u'James K. Polk'}
{'Average': 0.3820076114423448, 'President_2': u'Rutherford B. Hayes', 'President_1': u'

Reading the speeches of Barack Obama in 2012 & 2013 (easily the best similarity score among the '50 most similar pairs of SOUs given by the same President') we can see how themes of recession, jobs and the economy run rampant through both these speeches - highlighting one of the key aspects of Obama's presidency. With the word similarity analysis we just performed, it's easy to see how similar themes in respective speeches show higher similarity scores. They address similar issues that existed in America between 2012 & 2013. 

In order to get better similarity scores for this, we could've done a better job removing stop words (even though the vectorizer should've already removed them). Thus our speech analysis would've just encompassed words that pertain to the key points of the speech - giving out better similarity scores. Although I removed words that didn't occur that frequently, removing stop words would've been a great next step towards achieving better cosine similarity scores.

In [279]:
from sklearn.cluster import KMeans
n_clusters = 10
model = KMeans(n_clusters)
arr_clusters = sklearn_representation.toarray()
sou_clust=model.fit(arr_clusters)

In [280]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = sklearn_tfidf.get_feature_names()
for i in range(n_clusters):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

Top terms per cluster:
Cluster 0:  the  of  to  and  in  be  which  that  our  for
Cluster 1:  the  of  and  to  in  be  is  for  that  it
Cluster 2:  the  of  to  and  in  that  be  it  by  which
Cluster 3:  autocracy  the  of  1933  and  to  rulers  in  that  we
Cluster 4:  the  of  to  and  in  that  be  which  it  by
Cluster 5:  ve  re  tonight  ll  jobs  the  and  to  of  spending
Cluster 6:  the  of  and  to  in  that  for  be  by  is
Cluster 7:  tonight  ve  iraq  re  terrorists  the  and  to  americans  of
Cluster 8:  the  of  and  to  in  we  our  program  that  for
Cluster 9:  1942  hitler  japanese  nazis  1943  1941  axis  fighting  planes  tanks


Although stop words like 'the', 'and', 'of' and 'to' impede the purpose of the clusters, we can still see interpretable patterns develop. In Cluster 9, themes of World War II are clearly highlighted with the use of words 'tank', 'hitler', 'japanese', 'planes' and in Cluster 7, themes of the Iraq War and war on terrorism are easily interpretable. Themes of jobs (Cluster 5) and societal progress are also discernable as the clusters distinguish significant themes of SOU Speeches in America's History. 

In [293]:
from sklearn.cluster import KMeans
i = 0
n_clusters = 5
model = KMeans(n_clusters)
arr_clusters = sklearn_representation.toarray()
sou_clust=model.fit(arr_clusters)
i += 1

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = sklearn_tfidf.get_feature_names()
for i in range(n_clusters):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

Top terms per cluster:
Cluster 0:  the  of  to  and  in  be  which  that  our  by
Cluster 1:  the  of  to  and  in  that  be  for  by  it
Cluster 2:  tonight  the  and  to  of  ve  jobs  we  in  americans
Cluster 3:  the  of  and  to  in  be  is  for  that  it
Cluster 4:  the  of  and  to  in  we  that  our  is  for


With fewer clusters, it becomes tougher to distinguish the ideas and themes that separate each cluster as stop words flourish in our analysis. If I were to repeat this analysis I would remove the stop words from the nltk library. 